<a href="https://colab.research.google.com/github/truyentdm/NotebookTTS/blob/main/US%20Notebook%20TortoiseTTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Welcome to Tortoise! 🐢🐢🐢🐢

Before you begin, I **strongly** recommend you turn on a GPU runtime.

There's a reason this is called "Tortoise" - this model takes up to a minute to perform inference for a single sentence on a GPU. Expect waits on the order of hours on a CPU.

In [ ]:
# the scipy version packaged with colab is not tolerant of misformated WAV files.
# install the latest version.
!pip3 install -U scipy

!git clone https://github.com/jnordberg/tortoise-tts.git
%cd tortoise-tts
!pip3 install -r requirements.txt
!pip3 install transformers==4.19.0 einops==0.5.0 rotary_embedding_torch==0.1.5 unidecode==1.3.5
!python3 setup.py install

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.3/36.3 MB 53.6 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Uninstalling scipy-1.10.1:
      Successfully uninstalled scipy-1.10.1
Cloning into 'tortoise-tts'...
remote: Enumerating objects: 1481, done.
remote: Counting objects: 100% (408/408), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 1481 (delta 364), reused 359 (delta 359), pack-reused 1073
Receiving objects: 100% (1481/1481), 53.55 MiB | 29.62 MiB/s, done.
Resolving deltas: 100% (615/615), done.
/content/tortoise-tts
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Imports used through the rest of the notebook.
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F

import IPython

from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices

# This will download all the models used by Tortoise from the HuggingFace hub.
tts = TextToSpeech()

Downloading:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/159 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/181 [00:00<?, ?B/s]

...

Done.


100% (25193729 of 25193729) |############| Elapsed Time: 0:00:00 Time:  0:00:00
  0% (0 of 100715777) |                  | Elapsed Time: 0:00:00 ETA:  --:--:--

Done.


100% (100715777 of 100715777) |##########| Elapsed Time: 0:00:00 Time:  0:00:00


Done.


In [ ]:
#Tạo thư mục "result" và thư mục "holder":
import os
result_folder = './result'
holder_folder = './download'
voice_garyking = './tortoise/voices/GaryKing'
voice_NoShameIncome = './tortoise/voices/No Shame Income'
voice_RachaelXO = './tortoise/voices/Rachael XO'

# Tạo thư mục "result" nếu chưa tồn tại
if not os.path.exists(result_folder):
    os.makedirs(result_folder)

# Tạo thư mục "holder" nếu chưa tồn tại
if not os.path.exists(holder_folder):
    os.makedirs(holder_folder)
if not os.path.exists(voice_garyking):
    os.makedirs(voice_garyking)
if not os.path.exists(voice_NoShameIncome):
    os.makedirs(voice_NoShameIncome)
if not os.path.exists(voice_RachaelXO):
    os.makedirs(voice_RachaelXO)

Train Voice

In [ ]:
# Optionally, upload use your own voice by running the next two cells. I recommend
# you upload at least 2 audio clips. They must be a WAV file, 6-10 seconds long.
CUSTOM_VOICE_NAME = "martin"

import os
from google.colab import files

custom_voice_folder = f"tortoise/voices/{CUSTOM_VOICE_NAME}"
os.makedirs(custom_voice_folder)
for i, file_data in enumerate(files.upload().values()):
  with open(os.path.join(custom_voice_folder, f'{i}.wav'), 'wb') as f:
    f.write(file_data)

Saving clone 1.wav to clone 1.wav
Saving clone 2.wav to clone 2.wav
Saving clone 3.wav to clone 3.wav
Saving clone 4.wav to clone 4.wav
Saving clone 5.wav to clone 5.wav


Chạy Đơn Một đoạn

In [ ]:
# This is the text that will be spoken.
text = "Um, I actually found a pretty good deal on this, so I put a link below to that deal in the description below, so you can check that out if you're interested in picking one up for yourself."

# Pick a "preset mode" to determine quality. Options: {"ultra_fast", "fast" (default), "standard", "high_quality"}. See docs in api.py
preset = "fast"

In [ ]:
# Generate speech with the custotm voice.
voice_samples, conditioning_latents = load_voice(CUSTOM_VOICE_NAME)
gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents,
                          preset=preset)
torchaudio.save(f'result/generated-{CUSTOM_VOICE_NAME}.wav', gen.squeeze(0).cpu(), 48000)
IPython.display.Audio(f'result/generated-{CUSTOM_VOICE_NAME}.wav')

Generating autoregressive samples..


100%|██████████| 6/6 [00:48<00:00,  8.16s/it]


Computing best candidates using CLVP and CVVP


100%|██████████| 6/6 [00:12<00:00,  2.12s/it]


Transforming autoregressive outputs into audio..


100%|██████████| 80/80 [00:25<00:00,  3.08it/s]


Chạy một đoạn văn dài



In [ ]:
import shutil
from google.colab import files

def add_dots(current, max_length):
    if current < max_length:
        return "."
    else:
        return ""
def split_into_sentences(text):
    # Xóa hết dấu xuống dòng và dấu cách thừa trước khi tách câu
    cleaned_text = ' '.join(text.split())

    sentences = cleaned_text.split('. ')  # Tách câu bằng dấu chấm và khoảng trắng
    for i, sentence in enumerate(sentences):
        sentences[i] = sentence.strip()  # Loại bỏ khoảng trắng ở đầu và cuối câu
    return sentences
def merge_sentences(array, max_length):
    merged_array = []
    current_sentence = array[0]

    for i in range(1, len(array)):
        merged_text = current_sentence + '. ' + array[i]
        if len(merged_text) <= max_length:
            current_sentence = merged_text
        else:
            merged_array.append(current_sentence+add_dots(i,len(array)))
            current_sentence = array[i]

    if len(current_sentence) > 0:
        merged_array.append(current_sentence)

    return merged_array
text = """
Hello, everyone! Welcome to my channel. Today, I wanna do just kind of a quick review for you guys on the Novo full body Zero Gravity Massage chair by Human Touch. Over the past few weeks, I've had the pleasure of experiencing the comfort and luxury of this chair, and I can't wait to share my thoughts with you. Um, kind of based off my experience with it, and let you know what I thought about it.

Um, I should mention to you real quick that while this is a fairly expensive massage chair, I actually found a pretty good deal on it and I'll put a link in the description below so you can check that deal out if you're interested.

So, if you're like me, maybe you've sat in one of these full body massage chairs at like Brookstone or Sharper Image at the mall or something like that, and you really enjoyed that um, that's kind of, I guess you could say, my story. I had used one in uh, it was actually in Costco, um, really liked it, wanted to get one, and started doing what turned into quite a bit of online research into these things.

Um, there's a lot of bad information out there, there's a lot of good information out there too. There's a very wide disparity of pricing um, they charge, there's there's, let me put it this way, there's massagers out there that they charge like double the price of this one that aren't as good, there's ones out there that are half the price of this one that are really, really bad um, but to me it kind of came down to what's your best bang for your buck, and that's what the Novo is and I'm extremely happy with it and like I said, for the money, even though it is one of the more expensive units, it's an excellent value for what you're getting.

I'll tell you right off the get go if you go into like Sharper Image and the cost of this chair would be well over $20,000, that's why I don't recommend buying there um, when you can get it online so much cheaper, like literally a third of the price. But let me tell you a little bit more about the unit um, and what I like about it.

One, and kind of the most important thing, is the quality of the massage you get. I'm not saying that this particular chair could 100% um, get rid of like a human massage, meaning it can give you as good massages as a human being could, but man it is like 97% there, way beyond good enough for me where I'll never have to go spend $50 to $200 on a full body massage anymore. This chair is unbelievably good at giving massages, it has really all the bells and whistles that you can imagine. It has 30 automatic programs, um, it's zero G, which means it automatically reclines back and puts you in this kind of zero gravity position, it's basically just a recliner, I guess you could say, but for back and neck massages this thing is unbelievably good. They really took the time in their engineering to figure out how to give you a really, really good quality massage, so that to me is the most important thing.

Second thing I would say that I really like about this one is the build quality, um, that's kind of something you run into when you get into the cheaper models. Um, a lot of people, if you get into the sub $2,000 models, that's where the build quality starts to fall off on a lot of these units, um, they tend to break, they tend to wear out. This one is very, very solidly built. The motors they never feel like they're getting bogged down at all. It does come with a 5 year warranty, that's an in home labor warranty to where they actually come to your house and fix it for you or replace it if they can't fix it within 5 years, which is good to know um, but I just don't see this thing as ever really breaking, especially within 5 years or so. Um, it's just a very, very good quality unit, the leather they use on it is very high quality, just everything, it feels like a very uh, high end chair. Like I said, I would put it on par with models that I've used it like Sharper Image around the 20,000 dollar mark, um, it just feels like that good quality.

Now while it is a fairly complex unit, meaning you can do a lot of things, it has a remote with a touchscreen with an LCD screen on it, it's very simple to use, so don't let that put you off if you think it's all gonna be too difficult to figure out or use, it really isn't. You basically just pick your mode that you want, hit a button for it, and it'll do its thing. 99% of people are just gonna be using it for that. You can get real deep dive into the custom features, make custom programs for yourself and all that stuff if you want, but you can just simply use the remote for its one button programs and it works very well at that.

But yeah, I really can't recommend this massage chair enough, I really do like it a lot. If you suffer from back pain or anything like that, this thing is literally a God send. So highly recommended. If you have any more questions about it, just leave them in the comments below. Thanks for watching guys and have a great day."""
sentenceArray = split_into_sentences(text)

sentences = merge_sentences(sentenceArray,100)
#đảo ngược mảng
reversed_list = sentences[::-1]

def render_sentence(sentences,indexAV=0):
  print('INDEX: >>>> ',indexAV)
  if len(sentences) == 0:
    print("============================================================================================================")
    print('Finished Render...')
    return []
  #CUSTOM_VOICE_NAME: #GaryKing #No Shame Income #Rachael XO
  CUSTOM_VOICE_NAME = "No Shame Income"
  # Pick a "preset mode" to determine quality. Options: {"ultra_fast", "fast" (default), "standard", "high_quality"}. See docs in api.py
  preset = "fast"
  textRender = sentences.pop()
  # Generate speech with the custotm voice.
  voice_samples, conditioning_latents = load_voice(CUSTOM_VOICE_NAME)
  gen = tts.tts_with_preset(textRender, voice_samples=voice_samples, conditioning_latents=conditioning_latents,
                            preset=preset)
  torchaudio.save(f'result/generated-{CUSTOM_VOICE_NAME}-{indexAV}.wav', gen.squeeze(0).cpu(), 24000)
  #IPython.display.Audio(f'result/generated-{CUSTOM_VOICE_NAME}-{indexAV}.wav')
  # files.download(f'result/generated-{CUSTOM_VOICE_NAME}.wav')
  indexAV +=1
  render_sentence(sentences,indexAV)

print("LENGTH: ",str(len(reversed_list)))
render_sentence(reversed_list)

#Tải xuống thư mục

# Đường dẫn của thư mục bạn muốn nén
folder_path = './result'

# Đường dẫn của thư mục đã nén
output_path = './download/result'

# Nén thư mục
shutil.make_archive(output_path, 'zip', folder_path)

# Tạo liên kết tải xuống
files.download(output_path + '.zip')



LENGTH:  32
INDEX: >>>>  0
Generating autoregressive samples..


100%|██████████| 6/6 [00:11<00:00,  1.96s/it]


Computing best candidates using CLVP and CVVP


  0%|          | 0/6 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
100%|██████████| 6/6 [01:37<00:00, 16.30s/it]


Transforming autoregressive outputs into audio..


...

100%|██████████| 6/6 [01:38<00:00, 16.46s/it]


Transforming autoregressive outputs into audio..


100%|██████████| 80/80 [00:40<00:00,  1.99it/s]


INDEX: >>>>  18


KeyboardInterrupt: ignored

In [ ]:
#Tải xuống thư mục

# Đường dẫn của thư mục bạn muốn nén
folder_path = './result'

# Đường dẫn của thư mục đã nén
output_path = './download/result'

# Nén thư mục
shutil.make_archive(output_path, 'zip', folder_path)

# Tạo liên kết tải xuống
files.download(output_path + '.zip')

In [ ]:
# dọn dẹp thư mục
import os

def empty_folder(folder_path):
    try:
        items = os.listdir(folder_path)
        for item in items:
            item_path = os.path.join(folder_path, item)
            if os.path.isfile(item_path):
                os.remove(item_path)
            elif os.path.isdir(item_path):
                os.rmdir(item_path)
        print("Thư mục đã được làm rỗng:", folder_path)
    except Exception as e:
        print("Lỗi:", str(e))

empty_folder("./result")
empty_folder("./download")
#empty_folder("./tortoise/voices/No Shame Income")


Thư mục đã được làm rỗng: ./result
Thư mục đã được làm rỗng: ./download
